## Assignment: Working with Dependency Graphs (Parses)

The objective of the assignment is to learn how to work with dependency graphs by defining functions.

Read [spaCy documentation on dependency parser](https://spacy.io/api/dependencyparser) to learn provided methods.

Define functions to:

- extract a path of dependency relations from the ROOT to a token
    - input is a sentence, you parse it and get a Doc object of spaCy
    - for each token the path will be a list of dependency relations, where first element is ROOT
- extract subtree of a dependents given a token
    - input is a sentence, you parse it and get a Doc object of spaCy
    - for each token in Doc objects you extract a subtree of its dependents as a list (ordered w.r.t. sentence order)
- check if a given list of tokens (segment of a sentence) forms a subtree
    - you parse a sentence and get a Doc object of spaCy
    - providing as an input ordered list of words from a sentence, you output True/False based on the sequence forming a subtree or not
- identify head of a span, given its tokens
    - input is a sequence of words (not necessarily a sentence)
    - output is the head of the span (single word)
- extract sentence subject, direct object and indirect object spans
    - input is a sentence, you parse it and get a Doc object of spaCy
    - output is lists of words that form a span (not a single word) for subject, direct object, and indirect object (if present of course, otherwise empty)
        - dict of lists, is better

In [11]:
import spacy
spacy_nlp = spacy.load('en_core_web_sm')

# sentence = "The objective of the assignment is to learn how to work with dependency graphs by defining functions."
sentence = "Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29."
# sentence = "bright red apples on the tree"
# sentence = "Autonomous cars shift insurance liability toward manufacturers"
# sentence = "The teacher gave the class some homework."
# sentence = "Sue passed Ann the ball."

#display the dep graph
doc1 = spacy_nlp(sentence)
spacy.displacy.render(doc1, style='dep')

## First point

- extract a path of dependency relations from the ROOT to a token
    - input is a sentence, you parse it and get a Doc object of spaCy
    - for each token the path will be a list of dependency relations, where first element is ROOT

In this point my idea was to split the problem in two function:
- the first function __pathDep(token)__ return the path from the root to the token as a normal graph visiting. The __input__ is a *spaCy token* and the __output__ is a *list* with insite the touple *(dependecy_of_the_node, text_of_the_node)*
- the second function __printPathDep(sent)__ print all the paths of all the tokens inside the sentence. As __input__ a *string* representing the sentence

In [2]:
# visit as a normal graph visiting
def pathDep(token):
    S = []
    path = []
    S.append(token)
    while len(S) > 0:
        node = S.pop(0)
        path.append((node.dep_, node.text))
        if node.dep_ != "ROOT":
            S.append(node.head)
    return list(reversed(path))

def printPathDep(sent):
    doc = spacy_nlp(sent)
    path = []
    for token in doc:
        path = pathDep(token)
        print(path)

printPathDep(sentence)

[('ROOT', 'join'), ('nsubj', 'Vinken'), ('compound', 'Pierre')]
[('ROOT', 'join'), ('nsubj', 'Vinken')]
[('ROOT', 'join'), ('nsubj', 'Vinken'), ('punct', ',')]
[('ROOT', 'join'), ('nsubj', 'Vinken'), ('amod', 'old'), ('npadvmod', 'years'), ('nummod', '61')]
[('ROOT', 'join'), ('nsubj', 'Vinken'), ('amod', 'old'), ('npadvmod', 'years')]
[('ROOT', 'join'), ('nsubj', 'Vinken'), ('amod', 'old')]
[('ROOT', 'join'), ('nsubj', 'Vinken'), ('punct', ',')]
[('ROOT', 'join'), ('aux', 'will')]
[('ROOT', 'join')]
[('ROOT', 'join'), ('dobj', 'board'), ('det', 'the')]
[('ROOT', 'join'), ('dobj', 'board')]
[('ROOT', 'join'), ('prep', 'as')]
[('ROOT', 'join'), ('prep', 'as'), ('pobj', 'director'), ('det', 'a')]
[('ROOT', 'join'), ('prep', 'as'), ('pobj', 'director'), ('amod', 'nonexecutive')]
[('ROOT', 'join'), ('prep', 'as'), ('pobj', 'director')]
[('ROOT', 'join'), ('prep', 'as'), ('pobj', 'director'), ('npadvmod', 'Nov.')]
[('ROOT', 'join'), ('prep', 'as'), ('pobj', 'director'), ('npadvmod', 'Nov.')

## Second point

- extract subtree of a dependents given a token
    - input is a sentence, you parse it and get a Doc object of spaCy
    - for each token in Doc objects you extract a subtree of its dependents as a list (ordered w.r.t. sentence order)

Also in this point I decided to split the problem in two function, so that they can be used in the other points:
- the first function __subTreeDep(token)__ wants a sapCy token as input an return the list representing the subtree of the token (iterating over the token.subtree)
- the sencond function __printSubTreeDep(sent)__ recieve as input a sentence and print all the subtree of the tokens inside

In [3]:
def subTreeDep(token):
    tree = []
    for sub in token.subtree:
        tree.append(sub)
    return tree

def printSubTreeDep(sent):
    doc = spacy_nlp(sent)
    for token in doc:
        path = subTreeDep(token)
        print(path)

printSubTreeDep(sentence)

[Pierre]
[Pierre, Vinken, ,, 61, years, old, ,]
[,]
[61]
[61, years]
[61, years, old]
[,]
[will]
[Pierre, Vinken, ,, 61, years, old, ,, will, join, the, board, as, a, nonexecutive, director, Nov., 29, .]
[the]
[the, board]
[as, a, nonexecutive, director, Nov., 29]
[a]
[nonexecutive]
[a, nonexecutive, director, Nov., 29]
[Nov., 29]
[29]
[.]


## Third point

- check if a given list of tokens (segment of a sentence) forms a subtree
    - you parse a sentence and get a Doc object of spaCy
    - providing as an input ordered list of words from a sentence, you output True/False based on the sequence forming a subtree or not

In the third point the function __chekSubTree(sent, tokenList)__ get as input the *string* of the sentence to parse, and the *list* of tokens to check if they are contained in a subtree of the sentence. This function reuse *subTreeDep* to retreive the subtree of each token to then compare it with the passed token list. It returns a *Boolean* as __output__.

For testing I create a list of word from a sentence and then convert it in a sorted list of tokens.

In [9]:
# Pre-create a list of word from a sentence
tmp = "61 years old"
tokens = spacy_nlp(tmp)
listTokens = []
for t in tokens:
    listTokens.append(str(t))
listTokens.sort()

In [10]:
def checkSubTree(sent, tokenlist):
    doc = spacy_nlp(sent)
    for token in doc:
        path = subTreeDep(token)
        pathString = sorted([str(t) for t in path])
        if pathString == tokenlist:
            return True
    return False

print("Check if forms a subtree: {}".format(checkSubTree(sentence, listTokens)))

Check if forms a subtree: True


## Fourth point

- identify head of a span, given its tokens
    - input is a sequence of words (not necessarily a sentence)
    - output is the head of the span (single word)

Here my doubts was if the input of the function can be a normal *string* or a sliced spaCy document, for this reason I develop:

- __headSpan(span)__ recieve as __input__ a slice of a spaCy document and return the root

- __headSpanString(span)__ recieve as __input__ a string of list of word and then convert it do a spaCy document for the computing of the root

In [6]:
doc = spacy_nlp("Autonomous cars shift insurance liability toward manufacturers.")
def headSpan(span):
    return span.root.text

span = doc[3:6]
print("Span: {}".format(span))
print("Head of span: {}".format(headSpan(span)))

Span: insurance liability toward
Head of span: liability


In [7]:
def headSpanString(span):
    doc = spacy_nlp(span)
    return doc[0:len(doc)].root.text

print("Head of span: {}".format(headSpanString("insurance liability toward")))

Head of span: liability


## Fifth point

- extract sentence subject, direct object and indirect object spans
    - input is a sentence, you parse it and get a Doc object of spaCy
    - output is lists of words that form a span (not a single word) for subject, direct object, and indirect object (if present of course, otherwise empty)
        - dict of lists, is better

This function __extractDep(sent)__ recieve in input a *string* of the sentece, convert it as a spaCy doc and returns a *dictionary* containg all the span for the *subject*, *indirect object* and *direct object* by iterating in its subtree 

In [8]:
def extractDep(sent):
    doc = spacy_nlp(sent)
    dic = {}
    dic["nsubj"] = []
    dic["iobj"] = []
    dic["dobj"] = []
    
    for token in doc:
        if token.dep_ == "nsubj":
            for d in token.subtree:
                    dic["nsubj"].append(d.text)
        if token.dep_ == "iobj":
            for d in token.subtree:
                    dic["iobj"].append(d.text)
        if token.dep_ == "dobj":
            for d in token.subtree:
                    dic["dobj"].append(d.text)
    return dic

print(extractDep(sentence))

{'nsubj': ['Pierre', 'Vinken', ',', '61', 'years', 'old', ','], 'iobj': [], 'dobj': ['the', 'board']}
